In [4]:
import pandas as pd
import numpy as np
import boto3
from dotenv import load_dotenv
load_dotenv()
import os
import pymysql
import csv

In [5]:
# Create a connection to the MySQL database without SSL
conn = pymysql.connect(host=os.getenv("DB_HOST"), user=os.getenv("DB_USERNAME"), password=os.getenv("DB_PASSWORD"), ssl={'ssl_disabled': True})

In [6]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS customer_data (
    customer_id INT,
    credit_score INT,
    country VARCHAR(255),
    gender VARCHAR(255),
    age INT,
    tenure INT,
    balance DECIMAL(10, 2),
    products_number INT,
    credit_card INT,
    active_member INT,
    estimated_salary DECIMAL(10, 2),
    churn INT
); """

In [4]:
# Load data from a CSV file into the customer_data table
load_data_query = """
LOAD DATA LOCAL INFILE 'churn.csv' 
INTO TABLE customer_data 
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
IGNORE 1 LINES
(customer_id, credit_score, country, gender, age, tenure, balance, products_number, credit_card, active_member, estimated_salary, churn)
"""

In [7]:
get_Data = '''select * from customer_data limit 5;'''

In [8]:
access_key= os.getenv("access_key")
Secret_access_key =os.getenv("Secret_access_key")
region = os.getenv("region")

In [6]:
s3 = boto3.client('s3', aws_access_key_id=access_key,
                   aws_secret_access_key=Secret_access_key,
                   region_name=region)

bucket = 'luffydatalake'
key = 'temp/churn.csv'

with open('churn.csv', 'wb') as f:
    s3.download_fileobj(bucket, key, f)

In [14]:

cursor = conn.cursor()

# Execute the SQL command to create the table
cursor.execute(get_Data)
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
df.to_csv('query_results.csv', index=False)


In [18]:
csv_names = ['customer_profile_data','Distinct_Gender_Count','Distinct_Country_Count','Summary_Statistics',
             'Country_Credit_Score_Distribution','Churn_Effect_on_Financials_and_Gender','Age_Range_Churn_CreditCard_Insights',
             'Countrywise_Active_Member_Churn_Analysis','High_Balance_to_Salary_Customers','Low_Balance_to_Salary_Customers'
             ]

# Establish a connection to the MySQL database
conn = pymysql.connect(host=os.getenv("DB_HOST"), user=os.getenv("DB_USERNAME"), password=os.getenv("DB_PASSWORD"), ssl={'ssl_disabled': True})

# Read and execute SQL statements from the .sql file
with open('sql_to_table.sql', 'r') as sql_file:
    sql_statements = sql_file.read().split(';')

cursor = conn.cursor()

for i, sql_statement in enumerate(sql_statements):
    if sql_statement.strip():
        try:
            cursor.execute(sql_statement)
            result = cursor.fetchall()

            if result:
                # Retrieve data and store in a DataFrame
                df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])

                # Store the data in a CSV file
                csv_file_name = f'{csv_names[i]}.csv'
                df.to_csv(csv_file_name, index=False)
        except Exception as e:
            print(f"Error processing SQL statement {i}: {str(e)}")
            continue

# Close the connection
conn.close()

Error processing SQL statement 10: list index out of range


In [11]:
# CSV file to read
csv_file = 'churn.csv'

# SQL query to insert data into the database
insert_query = """
INSERT INTO customer_data (customer_id, credit_score, country, gender, age, tenure, balance, products_number, credit_card, active_member, estimated_salary, churn)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Open and read the CSV file, then insert the data into the database
with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row

    for row in csv_reader:
        cursor.execute(insert_query, row)

In [12]:
# Don't forget to close the cursor and connection when you're done
cursor.close()
conn.close()

In [13]:
df = pd.read_csv("churn.csv")

In [5]:
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
unique_tenure = df['tenure'].unique()
unique_tenure

array([ 2,  1,  8,  7,  4,  6,  3, 10,  5,  9,  0], dtype=int64)

In [5]:
df['balance_to_salary_ratio'] = np.where(df['balance'] == 0, 0, df['balance'] / df['estimated_salary'])

In [6]:
df["Credit_Utilization_Ratio"] = np.where(df['balance'] == 0, 0, df['balance'] / df['credit_score'])

In [7]:
score_ranges = [0, 600, 650, 700, 750, 850]
score_labels = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']

df['credit_score_range'] = pd.cut(df['credit_score'], bins=score_ranges, labels=score_labels)

In [8]:
unique_countries = df['country'].unique()
unique_countries

array(['France', 'Spain', 'Germany'], dtype=object)

In [9]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the "country" column
df['country'] = label_encoder.fit_transform(df['country'])


In [10]:
unique_countries = df['country'].unique()
unique_countries

array([0, 2, 1])

In [11]:
# Apply label encoding to the "country" column
df['gender'] = label_encoder.fit_transform(df['gender'])

In [12]:
# Apply label encoding to the "country" column
df['credit_score_range'] = label_encoder.fit_transform(df['credit_score_range'])

In [13]:
unique_gender = df['gender'].unique()
unique_gender

array([0, 1])

In [14]:
df.describe()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,balance_to_salary_ratio,Credit_Utilization_Ratio,credit_score_range
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,0.746300,0.545700,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,3.878703,120.301128,2.103500
std,7.193619e+04,96.653299,0.827529,0.497932,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,108.337260,101.294299,1.315128
min,1.556570e+07,350.000000,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,1.562853e+07,584.000000,0.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,1.000000
50%,1.569074e+07,652.000000,0.000000,1.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.747002,144.491235,2.000000
75%,1.575323e+07,718.000000,1.000000,1.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.514022,199.579503,3.000000
max,1.581569e+07,850.000000,2.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,10614.655440,464.804729,4.000000


In [15]:
max(df["credit_score"])

850

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
# Initialize the Min-Max scaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to numerical columns
numerical_columns = df.select_dtypes(include=['number']).columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Print the scaled DataFrame
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,balance_to_salary_ratio,Credit_Utilization_Ratio,credit_score_range
0,0.275616,0.538,0.0,0.0,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,1.0,0.000000,0.000000,0.25
1,0.326454,0.516,1.0,0.0,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0.0,0.000070,0.296559,0.25
2,0.214421,0.304,0.0,0.0,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,1.0,0.000132,0.684265,0.75
3,0.542636,0.698,0.0,0.0,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0.0,0.000000,0.000000,0.50
4,0.688778,1.000,1.0,0.0,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,0.0,0.000150,0.317681,0.00


In [18]:
new_df = df.drop('customer_id', axis=1)

In [19]:
y = df["churn"]
x = df.drop('churn', axis=1) 

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

In [23]:
# Initialize the GradientBoostingClassifier with your specified parameters
gbm = GradientBoostingClassifier(
    ccp_alpha=0.0,
    criterion='friedman_mse',
    init=None,
    learning_rate=0.1,
    loss='deviance',  # Use 'deviance' for classification tasks
    max_depth=3,
    max_features=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_samples_leaf=1,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_iter_no_change=None,
    random_state=694,
    subsample=1.0,
    tol=0.0001,
    validation_fraction=0.1,
    verbose=0,
    warm_start=False
)

# Train the GBM model on the training data
gbm.fit(X_train, y_train)

# Predict using the trained model
y_pred = gbm.predict(X_test)

C:\python39\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


In [24]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# Calculate predictions
y_pred = gbm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

# Calculate AUC
y_pred_prob = gbm.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"AUC: {auc:.2f}")


Accuracy: 0.87
F1-score: 0.57
Precision: 0.77
Recall: 0.46
AUC: 0.86


In [25]:
from pycaret.classification import *
churn = setup(df, target = 'churn', ignore_features = ['customer_id'])

,Description,Value
0,Session id,5093
1,Target,churn
2,Target type,Binary
3,Original data shape,"(10000, 15)"
4,Transformed data shape,"(10000, 14)"
5,Transformed train set shape,"(7000, 14)"
6,Transformed test set shape,"(3000, 14)"
7,Ignore features,1
8,Numeric features,13
9,Preprocess,True


In [26]:
# compare all models
best_df_churn_model = compare_models(sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8621,0.8625,0.4662,0.7667,0.5788,0.5023,0.5251,0.2720
lightgbm,Light Gradient Boosting Machine,0.8604,0.8581,0.5014,0.7283,0.5932,0.5126,0.5260,0.0550
ada,Ada Boost Classifier,0.8526,0.8495,0.4663,0.7121,0.5630,0.4789,0.4948,0.0890
rf,Random Forest Classifier,0.8599,0.8480,0.4439,0.7708,0.5632,0.4869,0.5132,0.2110
et,Extra Trees Classifier,0.8534,0.8439,0.4200,0.7530,0.5382,0.4594,0.4876,0.1530
xgboost,Extreme Gradient Boosting,0.8547,0.8379,0.4902,0.7067,0.5781,0.4940,0.5064,0.1690
qda,Quadratic Discriminant Analysis,0.8146,0.7960,0.2265,0.6240,0.3308,0.2501,0.2941,0.0150
nb,Naive Bayes,0.8186,0.7880,0.1929,0.7092,0.3000,0.2337,0.3010,0.0090
lda,Linear Discriminant Analysis,0.8061,0.7626,0.2040,0.5678,0.2999,0.2155,0.2539,0.0160
lr,Logistic Regression,0.8079,0.7594,0.1788,0.5994,0.2747,0.1995,0.2489,0.4980


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [27]:
# print best_model parameters
print(best_df_churn_model)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=5093, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [28]:
# tune best model
tuned_best_model_churn = tune_model(best_df_churn_model)
tuned_best_model_churn

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8700,0.8516,0.4615,0.8250,0.5919,0.5218,0.5530
1,0.8914,0.8913,0.5175,0.9136,0.6607,0.6019,0.6364
2,0.8657,0.8665,0.4545,0.8025,0.5804,0.5076,0.5367
3,0.8471,0.8466,0.4336,0.7045,0.5368,0.4514,0.4705
4,0.8700,0.8555,0.4965,0.7889,0.6094,0.5363,0.5570
5,0.8629,0.8528,0.4336,0.8052,0.5636,0.4908,0.5240
6,0.8586,0.8490,0.4507,0.7529,0.5639,0.4858,0.5086
7,0.8543,0.8456,0.4085,0.7632,0.5321,0.4550,0.4863
8,0.8671,0.8914,0.4507,0.8101,0.5792,0.5078,0.5386


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='log_loss', max_depth=2,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.3, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=260, n_iter_no_change=None,
                           random_state=5093, subsample=0.55, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)